In [ ]:
import sys
!{sys.executable} -m pip install polars praw

In [76]:
import re
import praw
import polars as pl

In [2]:

reddit = praw.Reddit(
    client_id="",
    client_secret="",
    user_agent="Comment Extraction (by u/emtee14_)",
)
print(reddit.read_only)

True


In [87]:
url = "https://www.reddit.com/r/CasualUK/comments/12jgwhq/what_you_going_for/"
submission = reddit.submission(url=url)

print("loading comments")
submission.comments.replace_more(limit=None)
print("loaded comments")

loading comments
loaded comments


In [88]:
combinations = []

combo_regex = re.compile(r"[A-f] ?[1-6] ?[A-f]")
for comment in submission.comments.list():
    try:
        reply = comment.body 
        combo = combo_regex.findall(reply)
        if combo != []:
            choices = combo[0].replace(" ", "").upper()
            combinations.append(
                list(choices) + [comment.score]
            )
    except KeyError:
        pass

In [97]:
df = pl.DataFrame(weighted_combinations, schema=["tea", "sandwich", "cake"], orient="row")
df.head()
len(df)


10700

In [90]:
popular_tea = df.groupby("tea").count().filter(pl.col("count") > 10).sort(by="count", descending=True)
popular_sandwich = df.groupby("sandwich").count().filter(pl.col("count") > 10).sort(by="count", descending=True)
popular_cakes = df.groupby("cake").count().filter(pl.col("count") > 10).sort(by="count", descending=True)

In [101]:
popular_combinations = df.with_columns([
            (pl.col("tea") + pl.col("sandwich") + pl.col("cake")).alias("combination")
]).groupby("combination").count().filter(pl.col("count") > 10).sort(by="count", descending=True).limit(10)
print(popular_combinations.head())

shape: (5, 2)
┌─────────────┬───────┐
│ combination ┆ count │
│ ---         ┆ ---   │
│ str         ┆ u32   │
╞═════════════╪═══════╡
│ C4B         ┆ 1562  │
│ C4A         ┆ 954   │
│ C4E         ┆ 919   │
│ C4C         ┆ 631   │
│ C2E         ┆ 605   │
└─────────────┴───────┘


In [93]:
popular_tea.head()

tea,count
str,u32
"""C""",7332
"""B""",1242
"""F""",1128
"""D""",625
"""E""",205


In [94]:
popular_sandwich.head()

sandwich,count
str,u32
"""4""",5328
"""3""",1630
"""2""",1339
"""1""",1253
"""5""",617


In [96]:
popular_cakes

cake,count
str,u32
"""B""",3119
"""A""",2542
"""E""",2355
"""C""",1472
"""F""",825
"""D""",376
